In [1]:
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, LSTM
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import model_from_json
import pandas as pd
import numpy as np
import tensorflow as tf

C:\Users\vales\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
own_df = pd.read_csv('D:/data/combo_df.csv', sep = ';')
shell_df = pd.read_csv('D:/data/shell_df.csv', sep = ';')

C:\Users\vales\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2683: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(own_df.columns)
own_df.head()

Index(['buyin', 'ownership', 'player_name', 'tournament_name', 'dk_type',
       'year', 'tournament_id', 'player_id', 'week_id', 'salary', 'date',
       'elapsed_days', 'dk_type_dummy', 'wins_last_30_events', 'result_0',
       'result_1', 'result_2', 'result_3', 'result_4', 'result_5', 'result_6',
       'result_7', 'result_8', 'result_9', 'own_0', 'own_1', 'own_2', 'own_3',
       'own_4', 'own_5', 'own_6', 'own_7', 'own_8', 'own_9', 'owgr_curr',
       'owgr_avg_3', 'owgr_avg_10', 'owgr_avg_25', 'owgr_avg_50',
       'num_tourny_appear', 'tourny_wins', 'tourny_cut_percent',
       'tourny_top_20_percent', 'appear_0', 'appear_1', 'appear_2', 'appear_3',
       'appear_4', 'appear_5', 'appear_6', 'appear_7', 'appear_8', 'appear_9'],
      dtype='object')


,buyin,ownership,player_name,tournament_name,dk_type,year,tournament_id,player_id,week_id,salary,...,appear_0,appear_1,appear_2,appear_3,appear_4,appear_5,appear_6,appear_7,appear_8,appear_9
0,5300,0.1053,Justin Thomas,Valspar Championship,GPP,2017,475.0,33448,2017_475_33448,11900.0,...,93.0,18.0,10.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,5300,0.6316,Henrik Stenson,Valspar Championship,GPP,2017,475.0,21528,2017_475_21528,11700.0,...,7.0,11.0,4.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,5300,0.0526,Matt Kuchar,Valspar Championship,GPP,2017,475.0,23108,2017_475_23108,10200.0,...,22.0,11.0,33.0,38.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,5300,0.0000,Daniel Berger,Valspar Championship,GPP,2017,475.0,40026,2017_475_40026,10100.0,...,70.0,11.0,72.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,5300,0.0526,Patrick Reed,Valspar Championship,GPP,2017,475.0,34360,2017_475_34360,9700.0,...,38.0,7.0,2.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [4]:
def replace_obj(x):
    try:
        return float(x.replace(',',''))
    except:
        return x


In [5]:
seed = 19

own_df.fillna(-999, inplace = True)
shell_df.fillna(-999, inplace = True)

own_df['buyin'] = own_df['buyin'].apply(lambda x: replace_obj(x))

train_cols = ['buyin', 'year', 'tournament_id', 'player_id', 'salary', 'elapsed_days', 'dk_type_dummy', 'wins_last_30_events',
             'result_0', 'result_1', 'result_2', 'result_3', 'result_4', 'result_5', 'result_6', 'result_7', 'result_8', 
             'result_9', 'own_0', 'own_1', 'own_2', 'own_3', 'own_4', 'own_5', 'own_6', 'own_7', 'own_8', 'own_9', 'owgr_curr',
             'owgr_avg_3', 'owgr_avg_10', 'owgr_avg_25', 'owgr_avg_50', 'num_tourny_appear', 'tourny_wins', 'tourny_cut_percent',
             'tourny_top_20_percent', 'appear_0', 'appear_1', 'appear_2', 'appear_3', 'appear_4', 'appear_5', 'appear_6', 
             'appear_7', 'appear_8', 'appear_9']



X = own_df[train_cols].as_matrix()
Y = own_df['ownership'].as_matrix()

X = shuffle(X, random_state = seed)
Y = shuffle(Y, random_state = seed)

shell_X = shell_df[train_cols].as_matrix()
shell_Y = shell_df['ownership'].as_matrix()


standard_scaler = StandardScaler()
standard_scaler_Y = StandardScaler()
standard_scaler.fit(X)
standard_scaler_Y.fit(Y)

X_scaled = standard_scaler.transform(X)
Y_scaled = standard_scaler_Y.transform(Y)

shell_scaled = standard_scaler.transform(shell_X)
shell_Y_scaled = standard_scaler_Y.transform(shell_Y)

input_size = len(train_cols)
layer_size = input_size * 80
layer_size_4 = round(layer_size / 5 * 4)
layer_size_3 = round(layer_size / 5 * 3)
layer_size_2 = round(layer_size / 5 * 2)
layer_size_1 = round(layer_size / 5)
batch_size = 2000
look_back = 3

C:\Users\vales\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\vales\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\vales\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single fea

In [29]:
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.65
#config.allow_soft_placement = True
#config.log_device_placement = True
#set_session(tf.Session(config = config))


#define base model
def baseline_model():
    model = Sequential()
    model.add(Dense(2000, input_shape = (input_size,), kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(Activation('linear'))
    #model.add(Dense(layer_size_4, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(2000, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(Activation('linear'))
    model.add(Dense(50, kernel_initializer = 'normal', activation = 'relu'))
    #model.add(Dense(layer_size_1, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer = 'normal'))
    
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    
    return model

In [35]:
seed = 19
np.random.seed(seed)

#estimators = []
#estimators.append(('standardize', StandardScaler()))
#estimators.append(('mlp', KerasRegressor(build_fn = baseline_model, epochs = 1000, batch_size = batch_size, verbose = 1)))
#pipeline = Pipeline(estimators)

model = KerasRegressor(build_fn = baseline_model, epochs = 20, batch_size = batch_size, verbose = 1,
                      validation_split = 0.35)
model.fit(X_scaled, Y_scaled)

#kfold = KFold(n_splits = 3, random_state = seed)
#results = cross_val_score(model, X_scaled, Y, cv = kfold)


#print('\nResults: {} mse {} mse_std'.format(results.mean(), results.std()))

Train on 27734 samples, validate on 14934 samples
Epoch 1/20
27734/27734 [==============================] - 1s - loss: 0.9332 - val_loss: 0.8060
Epoch 2/20
27734/27734 [==============================] - 1s - loss: 0.7143 - val_loss: 0.6774
Epoch 3/20
27734/27734 [==============================] - 1s - loss: 0.6392 - val_loss: 0.6372
Epoch 4/20
27734/27734 [==============================] - 1s - loss: 0.6005 - val_loss: 0.6092
Epoch 5/20
27734/27734 [==============================] - 1s - loss: 0.5690 - val_loss: 0.5568
Epoch 6/20
27734/27734 [==============================] - 1s - loss: 0.5294 - val_loss: 0.5379
Epoch 7/20
27734/27734 [==============================] - 1s - loss: 0.5119 - val_loss: 0.5126
Epoch 8/20
27734/27734 [==============================] - 1s - loss: 0.4859 - val_loss: 0.4939
Epoch 9/20
27734/27734 [==============================] - 1s - loss: 0.4694 - val_loss: 0.4623
Epoch 10/20
27734/27734 [==============================] - 1s - loss: 0.4443 - val_loss: 0.4477

In [15]:
model.model.save_weights('own_model.h5')

In [54]:
shell_predict = model.predict(shell_scaled)

1200/1960 [=================>............] - ETA: 0s

In [55]:
shell_df['predicted'] = shell_predict

shell_df.to_csv('D:/data/shell_predict1.csv', index = False)

In [59]:
model_json = model.model.to_json()
with open('own_model.json', 'w') as json_file:
    json_file.write(model_json)
    

#loaded_model = model_from_json(model_json)
loaded_model.load_weights('own_model.h5')

In [8]:
batch_size = 1200

def new_baseline_model():
    model = Sequential()
    model.add(Dense(layer_size + 20, input_shape = (input_size,), kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Activation('linear'))
    model.add(Dense(layer_size_4, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(layer_size_3, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(layer_size_2, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(layer_size_1, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer = 'normal'))
    model.load_weights('own_model.h5')
    
    
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    
    return model


new_model = KerasRegressor(build_fn = new_baseline_model, epochs = 1000, batch_size = batch_size, verbose = 1,
                      validation_split = 0.25)


new_model.fit(X_scaled, Y)

ValueError: Dimension 1 in both shapes must be equal, but are 1900 and 3780 for 'Assign_36' (op: 'Assign') with input shapes: [47,1900], [47,3780].

In [40]:
model.model.evaluate(X_scaled, Y_scaled)

42176/42668 [============================>.] - ETA: 0s

0.32858388920476045

In [9]:
Y

array([ 0.3514,  0.    ,  0.399 , ...,  0.03  ,  0.003 ,  0.005 ])